## Data preprocess

In [90]:
from pathlib import Path
from openpyxl import Workbook, load_workbook  # type: ignore
import os



def to_excel(
    data, filename, sheet_style, result_dir_path, new_sheet=False, sheet_name="Sheet1"
):

    if not os.path.exists(result_dir_path):
        os.makedirs(result_dir_path, exist_ok=True)

    file_path = os.path.join(result_dir_path, f"{filename}.xlsx")

    if os.path.exists(file_path):
        workbook = load_workbook(file_path)
        if new_sheet:
            sheet = workbook.create_sheet(title=sheet_name)

        else:
            sheet = workbook.active
    else:
        workbook = Workbook()
        default_sheet = workbook.active
        workbook.remove(default_sheet)
        sheet = workbook.create_sheet(title=sheet_name)
        
    if sheet_style:
        sheet.append(sheet_style)


    if new_sheet:
        print(data)
    # write data into file
    for row in data:
        sheet.append(row)

    workbook.save(filename=f"{result_dir_path}\\{filename}.xlsx")



# read input dataset
filename_prefix = (
    "RandomRequestNumberclientv_single_worker_node#loops3#requests_batch150#Thu-Aug-22-22-08-38-2024"
)


dataset_read_filename = filename_prefix
training_data_dir = Path.cwd().parent / "training_data" / "data_set8"


# Data preprocessing
file_path = f"{training_data_dir}\\{dataset_read_filename}.xlsx"



# set result output filename and path
result_suffix = "result"

result_dir_path = Path.cwd().parent / "results" / "result_processTime_waitTasks_v4"

if not os.path.exists(result_dir_path):
    os.makedirs(result_dir_path)

version_index = len([_ for _ in Path(result_dir_path).iterdir() if _.is_file()])
version = f"_v{version_index}"

result_name = "processTime#waitTasks" + version
result_output_filename = f"{filename_prefix}{result_name}{result_suffix}"

In [91]:

import pandas as pd # type: ignore
import numpy as np # type: ignore


def read_data(filename):
    df = pd.read_excel(filename)
    columns = df.columns.to_list()
    data_dict = {col: df[col].to_list() for col in columns}
    return data_dict


def data_preprocess(filepath):
    data = read_data(filepath)
    # TODO more...

    # to numpy
    for key in data.keys():
        data[key] = np.array(data[key])

    return data

In [92]:
dataset = data_preprocess(file_path)

## DATA Style View

In [93]:
# # dataset is a dictionary

print(type(dataset))

<class 'dict'>


## XGBOOST - 1

### Data split

In [94]:
from sklearn.model_selection import train_test_split # type: ignore
import cupy as cp

y = cp.array(dataset.get("processed_and_waited_time_on_manager_node"))



## Strandard Scaler

In [95]:
from sklearn.preprocessing import StandardScaler       # type: ignore
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
import numpy as np

_X = [dataset.get("request_num"), dataset.get("sub_processing_tasks_on_manager_node"), dataset.get("processed_time_on_worker_node")]

for i in range(len(_X)):
    _X[i] = np.array(_X[i]).reshape(-1, 1)
    

# _scaler = "normalization"
_scaler = None

if _scaler == "standard":
    X_scaler = StandardScaler()
    for i in range(len(_X)):
        _X[i] = X_scaler.fit_transform(_X[i])
elif _scaler == "normalization":
    X_scaler = MinMaxScaler()
    for i in range(len(_X)):
        _X[i] = X_scaler.fit_transform(_X[i])
elif _scaler == "log":
    for i in range(len(_X)):
        _X[i] = np.log1p(_X[i])
elif _scaler == "power":
    X_scaler = PowerTransformer(method='yeo-johnson')
    for i in range(len(_X)):
        _X[i] = X_scaler.fit_transform(_X[i])
else:
    pass

for i in range(len(_X)):
    _X[i] = _X[i].reshape(-1)

X = cp.asarray(np.array(
    _X
).T)

## XGBRegressor

In [96]:
import itertools

# Define the parameter ranges
params = {
    "max_depth": [5],
    "learning_rate": [0.01],
    "n_estimators": [200, 400],
    "min_child_weight": [1],
    "subsample": [0.1],
    "colsample_bytree": [0.1],
    "reg_alpha": [0.1],
    "reg_lambda": [0.1],
}


# Generate all combinations of parameters
parameter_combinations = [
    dict(zip(params.keys(), combination))
    for combination in itertools.product(*params.values())
]

# Print the number of parameter combinations and a few examples
print(f"Number of parameter combinations: {len(parameter_combinations)}")

Number of parameter combinations: 2


In [97]:
from xgboost import XGBRegressor  # type: ignore
import xgboost  # type: ignore
from sklearn.metrics import mean_squared_error, mean_absolute_error  # type: ignore
from typing import Tuple


# Create the XGBoost regressor model
def create_model(params):
    # if use gpu to add these two params
    # - tree_method="hist",
    # - device="cuda",
    
    return XGBRegressor(
        objective="reg:squarederror",
        n_estimators=params["n_estimators"],
        learning_rate=params["learning_rate"],
        max_depth=params["max_depth"],
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        reg_alpha=params["reg_alpha"],
        reg_lambda=params["reg_lambda"],
    )


# Function to evaluate model performance
def evaluate_model(
    params, X_train, y_train, X_test: np.array, y_test: np.array
) -> Tuple[np.float64, xgboost.Booster]:

    model = create_model(params)

    # Convert CuPy arrays to NumPy arrays

    model.fit(X_train, y_train, verbose=True)

    # Make predictions using the trained model
    predictions = model.predict(X_test)


    # typecasting  cp -> np
    y_test_np = cp.asnumpy(y_test)
    X_test_np = cp.asnumpy(X_test)
    predictions_np = cp.asnumpy(predictions)


    # Calculate mean squared error
    mse = float(mean_squared_error(y_test_np, predictions_np))
    mae = float(mean_absolute_error(y_test_np, predictions_np))
    avg = float(np.mean(y_test_np))
    avg_acc = 100 * (avg - mae) / avg

    # calculate diffirence and accuracy
    data_list = list()
    for i in range(len(y_test_np)):
        acc = 0
        diff = abs(y_test_np[i] - predictions_np[i])
        if predictions[i] < 0:
            acc = 0
        else:
            rate = diff / y_test_np[i]
            if rate < 1 and rate >= 0:
                acc = 1 - rate
            if rate > 1:
                acc = 0
        data_list.append(
            [
                float(X_test_np[i][0]),
                float(X_test_np[i][1]),
                float(X_test_np[i][2]),
                float(y_test_np[i]),
                float(predictions_np[i]),
                float(diff),
                float(round(acc, 5)),
            ],
        )

    results_tuple = {"mse": mse, "mae": mae, "avg": avg, "avg_acc": avg_acc, "model": model, "data_list": data_list}
    filtered_results = {k: v for k, v in results_tuple.items() if k != "model" and k != "data_list"}
    print(filtered_results, f'\n{params}', '\n')
    return results_tuple

In [98]:
# Initialize variables to track the best parameters
best_params = None
best_result = None

# results param
best_mse = float("inf")
best_avg_acc = float(-1)
best_mae = float("inf")

# result params tuple
best_avg_acc_params = None
best_mae_params = None


# control sheet style of excel written
first_write = True



# Evaluate each parameter combination
for params in parameter_combinations:

    # split dataset to train dataset and test dataset for every params group
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=None
    )
    results_tuple = evaluate_model(params, X_train, y_train, X_test, y_test)

    filtered_results = {
        k: v for k, v in results_tuple.items() if k != "model" and k != "data_list"
    }

    # update best model
    if results_tuple['mse'] < best_mse:
        best_mse = results_tuple["mse"]
        best_params = params
        best_model = results_tuple["model"]
        best_result = filtered_results

    # write result into excel file
    sheet_style = ["num", "jobs", "processed_time", "X_test", "prediction", "difference", "accuracy"]
    avg_acc = results_tuple["avg_acc"]
    data_list = results_tuple['data_list']

    if best_avg_acc < results_tuple['avg_acc']:
        best_avg_acc = results_tuple["avg_acc"]
        best_acc_params = params

    if best_mae > results_tuple['mae']:
        best_mae = results_tuple["mae"]
        best_mae_params = params

    if first_write:
        # sheet 1
        to_excel(
            data=data_list,
            filename=result_output_filename,
            sheet_style=sheet_style,
            result_dir_path=result_dir_path,
            new_sheet=False,
        )
        first_write = False
    else:
        to_excel(
            data=data_list,
            filename=result_output_filename,
            sheet_style=None,
            result_dir_path=result_dir_path,
            new_sheet=False,
        )
    print(f"Best acc {best_avg_acc}")

print("Best Parameters:", best_params)
print("Best MSE:", best_mse)
print("Best MAE:", best_result["mae"])
print("Average:", best_result["avg"])
print("Average accuracy:", best_result["avg_acc"])
print("Results saved finished.")

params_keys_list = list(best_params.keys())
params_values_list = list(best_params.values())

# best mse params
to_excel(
    data=[params_values_list],
    filename=result_output_filename,
    sheet_style=params_keys_list,
    result_dir_path=result_dir_path,
    new_sheet=True,
    sheet_name="best mse params",
)


result_keys_list = list(filtered_results.keys())

# avg_acc sheet
to_excel(
    data=[list(best_acc_params.values())],
    filename=result_output_filename,
    sheet_style=result_keys_list,
    result_dir_path=result_dir_path,
    new_sheet=True,
    sheet_name="best avg acc params",
)


# mae sheet
to_excel(
    data=[list(best_mae_params.values())],
    filename=result_output_filename,
    sheet_style=result_keys_list,
    result_dir_path=result_dir_path,
    new_sheet=True,
    sheet_name="best mae params",
)

{'mse': 558.5609927744022, 'mae': 20.41687072912852, 'avg': 53.12526133590274, 'avg_acc': 61.56843238843414} 
{'max_depth': 5, 'learning_rate': 0.01, 'n_estimators': 200, 'min_child_weight': 1, 'subsample': 0.1, 'colsample_bytree': 0.1, 'reg_alpha': 0.1, 'reg_lambda': 0.1} 

Best acc 61.56843238843414
{'mse': 516.4825417899154, 'mae': 19.014944619602627, 'avg': 46.475114422374304, 'avg_acc': 59.085749748152644} 
{'max_depth': 5, 'learning_rate': 0.01, 'n_estimators': 400, 'min_child_weight': 1, 'subsample': 0.1, 'colsample_bytree': 0.1, 'reg_alpha': 0.1, 'reg_lambda': 0.1} 

Best acc 61.56843238843414
Best Parameters: {'max_depth': 5, 'learning_rate': 0.01, 'n_estimators': 400, 'min_child_weight': 1, 'subsample': 0.1, 'colsample_bytree': 0.1, 'reg_alpha': 0.1, 'reg_lambda': 0.1}
Best MSE: 516.4825417899154
Best MAE: 19.014944619602627
Average: 46.475114422374304
Average accuracy: 59.085749748152644
Results saved finished.
[[5, 0.01, 400, 1, 0.1, 0.1, 0.1, 0.1]]
[[5, 0.01, 200, 1, 0.1, 

## Model save

In [99]:
# from pathlib import Path
# model_file = "xboost_tasks_time" + ".json"
# model_path = str(Path.cwd() / "modelsfile" / model_file)
# best_model.save_model(model_path)